In [3]:
# Import Numpy, Pandas and norm from Scipy stats

import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf

In [78]:
# American Option Function for Call option using Least Squares Method by Longstaff Schwartz

def MCS_A(S0=100, rf=0.05, T=1, K=100, vol=0.2, steps=252, sims=10000): 
    # Steps are equal to days in a time period to expiry. It can be smaller or larger 
    # dt in this case is 1 day. This is the time interval for each steps in the simulation
    dt = T/steps 
    # Creating a zero filled array of size = sims x steps + 1. This will be simulation of price paths taken
    S = np.zeros((sims, steps + 1)) 
    # The first price for each path is S0
    S[:,0] = S0
    # All the steps for all the paths can now be filled using a 'for' loop implementing GBM
    for i in range (steps):
        # phi consists of an 1D array of standard normal random numbers of length sims
        phi = np.random.randn(sims)
        # using GBM formula, create path wise prices. phi is the proxy for Z
        S[:, i+1] = S[:, i ] * np.exp((rf - 0.5 * vol * vol) * dt + vol * phi * np.sqrt(dt))
    S = pd.DataFrame(S)
    # Payoff Matrix is the intrinsic value of the option at every step for every path
    A = np.zeros((sims, steps + 1))
    A = pd.DataFrame(A)
    
    A.iloc[:,steps] = np.maximum(S.iloc[:,steps] - K,0)
    for i in range(steps - 1,0,-1):
        A.iloc[:,i] = np.maximum(S.iloc[:,i] - K,0)
    A = pd.DataFrame(A)
    
    # Valuation Matrix using backward induction
    C = np.zeros((sims, steps + 1))
    C = pd.DataFrame(C)
    # At expiry, Value of Option is the payoff
    C.iloc[:,steps] = A.iloc[:,steps]
    # For every other steps, Value shall be maximum of 'Continuation Value' and 'Option Payoff'
    # Continuation Value shall be the discounted value of option in the next step
    # Discount Factor
    dfactor = np.exp(-rf * dt)
    
    for i in range(steps - 1,0,-1):
        # First we will assign Value as the Continuation Factor
        C.iloc[:,i] = C.iloc[:,i + 1] * dfactor
        originalC = pd.DataFrame(C.iloc[:,i].copy())
        # The conditional expection of continuation will be determined using Least Squares (LS)
        # For LS, only those paths that contain a positive payoffs will be chosen
        positivePayoffPaths = A.loc[A[i]>0,].index
        # X values are Asset Prices for the paths
        xVal = S.loc[positivePayoffPaths,i]
        # Y values are Continuation Value for the paths
        yVal = C.loc[positivePayoffPaths,i]
        # Polynomial fitting with order of 2 as per Longstaff Schwartz
        reg = np.polyfit(xVal,yVal,2)
        # Recompute Continuation Value using the regression function
        revisedC = np.polyval(reg,xVal)     
        # Substitute revised Continuation Value to Value Matrix
        originalC.loc[positivePayoffPaths, i] = revisedC
        # Final value for the step is computed as maximum of 'Continuation Value' and 'Option Payoff'
        C.iloc[:,i].where(A.iloc[:,i]>originalC.iloc[:,0],A.iloc[:,i])     
    # Discounting the value of the option in first step to T0    
    C.iloc[:,0] = C.iloc[:,1] * dfactor
    # Option Value is the average of the discounted values on T0
    OptionVal = C[0].mean(axis = 0)
    
    return OptionVal



In [79]:
S0 = 100
rf = 0.01
T = 1
K = 100
vol = 0.20
steps = 252
sims = 1000

np.random.seed(40) 

# output array of price S at time T loaded to a dataframe
CA_MCS = MCS_A(S0, rf, T, K, vol, steps, sims)
print(CA_MCS)

8.658691162967619
